#

리턴값은 항상

ko,en

In [59]:
#import keyboard # 키보드 입력 감지를 위한 라이브러리

In [60]:
from pynput.keyboard import Controller as KeyCtrl
from pynput.keyboard import Key

In [61]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import pandas as pd
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt
#import keyboard # 키보드 입력 감지를 위한 라이브러리
import ctypes
import time
from ctypes import wintypes
import time

# Define the allowed characters (26 lowercase + 26 uppercase)
ALPHABET = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
char_to_idx = {char: idx for idx, char in enumerate(ALPHABET)}

def one_hot_encode(letter):
    """Return a 52-dim one-hot vector for a given letter."""
    vec = np.zeros(len(ALPHABET), dtype=np.float32)
    if letter in char_to_idx:
        vec[char_to_idx[letter]] = 1.0
    return vec
class LanguageClassifier(nn.Module):
    def __init__(self, input_size=52, hidden_size=128, num_layers=1):
        super(LanguageClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, lengths):
        # Pack the padded sequence
        packed = nn.utils.rnn.pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        packed_out, (h_n, _) = self.lstm(packed)
        # h_n is of shape (num_layers, batch, hidden_size); we use the last layer’s hidden state
        h_last = h_n[-1]  # shape: (batch, hidden_size)
        out = self.fc(h_last)  # shape: (batch, 1)
        out = self.sigmoid(out)  # Output between 0 and 1
        return out


model = LanguageClassifier(input_size=52, hidden_size=128, num_layers=1)

try:
    state_dict = torch.load("AI/best_model.pt")
    load_info = model.load_state_dict(state_dict, strict=False) # Keep strict=False for compatibility info

    if load_info.missing_keys or load_info.unexpected_keys:
        print(f"State dict has missing or unexpected keys.")
    else:
        print("Model loaded successfully.")

except FileNotFoundError:
    print("Model file not found.")
    # Handle file not found (e.g., initialize model differently)

except Exception as e:
    print(f"Error loading model: {e}")
    # Handle other loading errors

else:
    print("Loading process complete.") # Optional success log


def predict_language(input_str):
    model.eval()
    seq = [one_hot_encode(ch) for ch in input_str if ch in char_to_idx]
    if not seq: return 0.5  # 기본값
    
    seq_tensor = torch.tensor(seq).unsqueeze(0)
    length = torch.tensor([len(seq)])
    
    with torch.no_grad():
        output = model(seq_tensor, length)
    return output.item()

Model loaded successfully.
Loading process complete.


In [62]:
#### check if this works later
# import platform
# import os
# import ctypes


# def switch_layout(target_lang):
#     if platform.system() == 'Windows':
#         # HKL_US = 0x04090409, HKL_Korean = 0x04120412
#         lang_code = 0x04120412 if target_lang == "ko" else 0x04090409
#         ctypes.windll.user32.ActivateKeyboardLayout(lang_code, 0)
#     elif platform.system() == 'Darwin':
#         src = """
#         # macOS 입력 소스 전환 코드 (Objective-C)
#         """
#         os.system(f"osascript -e '{src}'")

In [63]:
from process.KoEnMapper import conv_en2ko, conv_ko2en

def process_conversion(new_layout):
    global buffered_word, converted_positions
    
    if new_layout == "ko":
        converted = conv_en2ko(buffered_word)
    if new_layout == "en":
        converted = conv_ko2en(buffered_word)
    
    # 백스페이스 시뮬레이션
    physical_keyboard.press(Key.backspace)
    physical_keyboard.release(Key.backspace)
    
    # 변환 텍스트 입력
    physical_keyboard.type(converted)
    converted_positions.add(len(buffered_word))

In [64]:
import subprocess
import sys
import platform
from pynput import keyboard
from datetime import datetime
import time
import ctypes  # Needed for Windows API calls
from langdetect import detect, LangDetectException
import os

# OS 감지
IS_MAC = sys.platform == 'darwin'
IS_WIN = sys.platform.startswith('win')
IS_LINUX = sys.platform.startswith('linux')

print(sys.platform)

if IS_MAC:
    print("MacOS")
    def switch_language():
        # Control 키와 Space 키를 조합하여 전환
        command = """
        osascript -e 'tell application "System Events"
            key down control
            delay 0.000
            key down space
            delay 0.000
            key up space
            delay 0.000
            key up control
        end tell'
        """
        subprocess.run(command, shell=True)
    # LEGACY CODE (checks the current input source by reading the system preferences)
    def check_language_mode():
        # MacOS의 현재 입력 소스 확인
        command = "defaults read ~/Library/Preferences/com.apple.HIToolbox.plist AppleSelectedInputSources"
        result = subprocess.run(command, shell=True, capture_output=True, text=True)
        input_source_data = result.stdout
        # English (ABC) 레이아웃 체크
        if '"KeyboardLayout Name" = ABC' in input_source_data or 'KeyboardLayout Name = ABC' in input_source_data:
            return 'en'
        # Korean 레이아웃 체크
        elif '"Bundle ID" = "com.apple.inputmethod.Korean"' in input_source_data or 'Bundle ID = com.apple.inputmethod.Korean' in input_source_data:
            return 'ko'
        else:
            return 'unknown'

elif IS_WIN:

    exe_path = "CheckCurrentLanguage.exe"
    # ULONG_PTR 타입 정의
    wintypes.ULONG_PTR = wintypes.WPARAM

    # User32.dll 로드
    hllDll = ctypes.WinDLL ("User32.dll", use_last_error=True)

    # 한/영 키 Virtual Key 코드 (VK_HANGUEL)
    VK_HANGUEL = 0x15

    # INPUT 구조체 정의
    class MOUSEINPUT(ctypes.Structure):
        _fields_ = (("dx",          wintypes.LONG),
                    ("dy",          wintypes.LONG),
                    ("mouseData",   wintypes.DWORD),
                    ("dwFlags",     wintypes.DWORD),
                    ("time",        wintypes.DWORD),
                    ("dwExtraInfo", wintypes.ULONG_PTR))

    class HARDWAREINPUT(ctypes.Structure):
        _fields_ = (("uMsg",    wintypes.DWORD),
                    ("wParamL", wintypes.WORD),
                    ("wParamH", wintypes.WORD))

    class KEYBDINPUT(ctypes.Structure):
        _fields_ = (("wVk",         wintypes.WORD), # Virtual-key code
                    ("wScan",       wintypes.WORD), # Hardware scan code
                    ("dwFlags",     wintypes.DWORD),# 플래그
                    ("time",        wintypes.DWORD),# 시간
                    ("dwExtraInfo", wintypes.ULONG_PTR))# 추가 정보

    class INPUT(ctypes.Structure):
        class _INPUT(ctypes.Union):
            _fields_ = (("ki", KEYBDINPUT),
                        ("mi", MOUSEINPUT),
                        ("hi", HARDWAREINPUT))
        _anonymous_ = ("_input",) # 익명 유니온
        _fields_ = (("type",   wintypes.DWORD), # 입력 타입
                    ("_input", _INPUT))       # 유니온 구조체

    # 한/영 상태 변경 함수
    def switch_language():
        x = INPUT(type=1 ,ki=KEYBDINPUT(wVk=VK_HANGUEL)) # 키 누름 이벤트
        y = INPUT(type=1, ki=KEYBDINPUT(wVk=VK_HANGUEL,dwFlags=2)) # 키 뗌 이벤트
        hllDll.SendInput(1, ctypes.byref(x), ctypes.sizeof(x)) # 키 누름 이벤트 전송
        time.sleep(0.05) # 딜레이
        hllDll.SendInput(1, ctypes.byref(y), ctypes.sizeof(y)) # 키 뗌 이벤트 전송
    def check_language_mode():
        """
        Executes the CheckCurrentLanguage.exe and returns the language code.

        Args:
            exe_path: The full path to the CheckCurrentLanguage.exe file.

        Returns:
            "ko" for Korean, "en" for English, "unknown" for others, or None if an error occurred.
        """
        try:
            if not os.path.exists(exe_path):
                return "Error, files does not exsist"

            result = subprocess.run([exe_path], capture_output=True, text=True, check=True)
            output = result.stdout.strip()

            if "Korean input mode" in output:
                return "ko"
            elif "English (US) keyboard layout detected" in output:
                return "en"
            elif "Detected language: ko" in output:
                return "ko"
            elif "Detected language: en" in output:
                return "en"
            elif "Detected language:" in output:
                return "unknown"
            elif "No foreground window detected" in output:
                return "en"
            elif "Korean keyboard layout detected, but no IME window found." in output:
                return "en"
            elif "Korean keyboard layout detected, but Korean IME mode is inactive." in output:
                return "en"
            else:
                return "unknown"

        except (subprocess.CalledProcessError, FileNotFoundError, Exception):
            return None

elif IS_LINUX:
    def switch_language():
        # Toggle between English and Korean layouts.
        current = check_language_mode()
        if current == 'en':
            new_layout = 'kr'
        else:
            new_layout = 'us'
        command = f"setxkbmap {new_layout}"
        subprocess.run(command, shell=True)

else:
    print("Error: Unsupported OS. This script supports only macOS, Windows, and Linux.")

win32


In [65]:
def replace_buffred_word(new_word,current_layout):
    ko_new_word = new_word
    #-- backspace simulation -- 
    if current_layout == "ko":# It is needed to measure the length of the buffered word
        new_word = conv_en2ko(new_word)
        #print(new_word)
    needed_backspace = len(new_word)
    #print(needed_backspace)
    if IS_MAC:
        for _ in range(needed_backspace):
            physical_keyboard.press(Key.backspace)
            physical_keyboard.release(Key.backspace)
    if IS_WIN:
        for _ in range(needed_backspace):
            physical_keyboard.press(Key.backspace)
            physical_keyboard.release(Key.backspace)
    #-- replace the buffered word --
    if current_layout == "ko":
        print("ko New word")
        print(ko_new_word)
        replace_word = ko_new_word
        switch_layout("en")
        #print(replace_word)
    elif current_layout == "en":
        replace_word = conv_en2ko(new_word)
        switch_layout("ko")
        #print(replace_word)
    physical_keyboard.type(replace_word)
    return replace_word

In [66]:
# current_layout = "en"
# replace_buffred_word("dlrjs gksrnrdj",current_layout)

In [67]:
# 상수 및 전역 변수 설정
THRESHOLD_HIGH = 0.95  # 영어 전환 임계값
THRESHOLD_LOW = 0.05   # 한글 전환 임계값
buffered_word = ""
physical_keyboard = KeyCtrl()
converted_positions = set()
current_layout = "Other"

이 코드 수정할점
1. f1을 누르면 일시정지 / 재생 할것
2. f4를 누르면 프로그램이 정지될것.
3. 아무 키를 누를때마다 현재 키보드의 언어를 확인해서 변경됬을 경우에 ui에 업데이트 할것
4. 현재 문제점이 버퍼가 n개만큼 남아야함. 만약에 n개를 넘어가면 옆으로 밀면서 지울것
5. 기본값을 "space"를 누를때 예측이 아니라 every N keystoke 로 할것
6. 이 코드에 대해서 피드백 해줘
7. 코드를 더 깔끔하게 만들것 

In [68]:
# import tkinter as tk
# from tkinter import ttk
# from pynput import keyboard
# from datetime import datetime

# current_layout = "en"  # Default layout: English
# THRESHOLD_LOW = 0.01  # Threshold for switching to Korean
# THRESHOLD_HIGH = 0.99  # Threshold for switching to English
# log_file = 'key_event_log.txt'  # Log file path

# # Logging function
# def log_message(message):
#     """Append a message to the log file with a timestamp."""
#     timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
#     with open(log_file, 'a', encoding='utf-8') as f:
#         f.write(f"{timestamp} - {message}\n")

# # Layout switching function
# def switch_layout(new_layout):
#     """Switch the keyboard layout and convert the buffer."""
#     global current_layout, buffered_word
#     if new_layout == current_layout:
#         return

#     if new_layout == "ko":
#         converted_text = conv_en2ko(buffered_word)
#         print(f"Buffer conversion (Eng->Kor): '{buffered_word}' -> '{converted_text}'")
#         #log_message(f"Switched to Korean layout. Buffer: '{buffered_word}' -> '{converted_text}'")
#     elif new_layout == "en":
#         converted_text = conv_ko2en(buffered_word)
#         print(f"Buffer conversion (Kor->Eng): '{buffered_word}' -> '{converted_text}'")
#         #log_message(f"Switched to English layout. Buffer: '{buffered_word}' -> '{converted_text}'")

#     buffered_word = ""
#     switch_language()
#     current_layout = new_layout
#     print(f"Keyboard layout switched to {current_layout}")
#     app.update_status()

# # UI Application Class
# class KeyboardPredictorApp:
#     def __init__(self, root):
#         """Initialize the Tkinter UI."""
#         self.root = root
#         root.title("Keyboard Predictor")

#         # Prediction trigger options
#         self.option_var = tk.StringVar(value="space")
#         ttk.Label(root, text="Prediction Trigger:").grid(row=0, column=0, padx=5, pady=5)
#         ttk.Radiobutton(root, text="On Spacebar", variable=self.option_var, value="space").grid(row=0, column=1, padx=5, pady=5)
#         ttk.Radiobutton(root, text="Every N keystrokes", variable=self.option_var, value="keystrokes").grid(row=0, column=2, padx=5, pady=5)

#         # Keystroke count input
#         self.n_var = tk.IntVar(value=5)  # Default: 5 keystrokes
#         ttk.Label(root, text="N keystrokes:").grid(row=1, column=0, padx=5, pady=5)
#         ttk.Entry(root, textvariable=self.n_var, width=5).grid(row=1, column=1, padx=5, pady=5)

#         # Clear buffer button
#         ttk.Button(root, text="Clear Buffer", command=self.clear_buffer).grid(row=2, column=0, columnspan=3, pady=10)

#         # Status label
#         self.status_var = tk.StringVar(value="Current Layout: English")
#         ttk.Label(root, textvariable=self.status_var).grid(row=3, column=0, columnspan=3, pady=5)

#         # Start keyboard listener
#         self.keystroke_count = 0
#         self.listener = keyboard.Listener(on_press=self.on_press)
#         self.listener.start()

#     def on_press(self, key):
#         """Handle key press events."""
#         global buffered_word, current_layout

#         try:
#             # Handle spacebar
#             if key == keyboard.Key.space:
#                 if self.option_var.get() == "space":
#                     buffered_word = ""
#                     self.keystroke_count = 0
#                     self.predict_and_switch()
#                 else:
#                     buffered_word += " "
#                     self.keystroke_count += 1
#                     if self.keystroke_count >= self.n_var.get():
#                         self.predict_and_switch()
#                         self.keystroke_count = 0 # Reset keystroke count                
#                 return

#             # Handle backspace
#             if key == keyboard.Key.backspace:
#                 if buffered_word:
#                     buffered_word = buffered_word[:-1]
#                     print(f"Backspace pressed. Buffer: '{buffered_word}'")
#                 return

#             # Handle character keys
#             if hasattr(key, 'char') and key.char:
#                 current_layout = check_language_mode()
#                 #print("current layout: ",current_layout)
#                 char = key.char
#                 if char.isalpha() or char.isdigit() or char in "!@#$%^&*()_+-=[]{}|;:'\",.<>/?`~":
#                     if current_layout == "ko":
#                         char = conv_ko2en(char)  # Adjust if in Korean mode
#                     buffered_word += char
#                     self.keystroke_count += 1
#                     log_message(f"Key pressed: '{char}', Buffer: '{buffered_word}'")

#                     if self.option_var.get() == "keystrokes" and self.keystroke_count >= self.n_var.get():
#                         self.predict_and_switch()
#                         self.keystroke_count = 0

#         except AttributeError:
#             pass

#     def predict_and_switch(self):
#         """Predict language and switch layout if needed."""
#         global buffered_word, current_layout

#         prob = predict_language(buffered_word)
#         print(f"LSTM prediction probability: {prob:.5f} (Eng=1, Kor=0)")
#         if current_layout == "en" and prob <= THRESHOLD_LOW:
#             #print("Detected Korean. Switching to Korean layout.")
#             replace_buffred_word(buffered_word, current_layout)
#             switch_layout("ko")
#         elif current_layout == "ko" and prob >= THRESHOLD_HIGH:
#             #print("Detected English. Switching to English layout.")
#             replace_buffred_word(buffered_word, current_layout)
#             switch_layout("en")

#     def clear_buffer(self):
#         """Clear the buffer and reset keystroke count."""
#         global buffered_word
#         buffered_word = ""
#         self.keystroke_count = 0
#         print("Buffer cleared.")
#         log_message("Buffer cleared via button.")

#     def update_status(self):
#         """Update the status label with the current layout."""
#         global current_layout
        
#         if current_layout == 'en':
#             self.status_var.set("Current Layout: English")
#         elif current_layout == 'ko':
#             self.status_var.set("Current Layout: Korean")
#         else:
#             self.status_var.set("Current Layout: Unknown")

# # Main execgg
# if __name__ == "__main__":
#     root = tk.Tk()
#     app = KeyboardPredictorApp(root)
#     root.mainloop()

In [69]:
import tkinter as tk
from tkinter import ttk
from pynput import keyboard
from datetime import datetime
from collections import deque


THRESHOLD_LOW = 0.1
THRESHOLD_HIGH = 0.8
log_file = 'key_event_log.txt'

class KeyboardPredictorApp:
    def __init__(self, root):
        self.root = root
        root.title("Keyboard Predictor")
        
        # 상태 변수 초기화
        self.current_layout = "en"
        self.buffered_word = ""
        self.paused = False
        self.keystroke_count = 0

        # UI 구성
        self.setup_ui()
        
        # 키보드 리스너 시작
        self.listener = keyboard.Listener(on_press=self.on_press)
        self.listener.start()

    def setup_ui(self):
        """UI 구성 요소 초기화"""
        self.option_var = tk.StringVar(value="keystrokes")
        
        ttk.Label(self.root, text="Prediction Trigger:").grid(row=0, column=0, padx=5, pady=5)
        ttk.Radiobutton(self.root, text="On Spacebar", variable=self.option_var, value="space").grid(row=0, column=1, padx=5, pady=5)
        ttk.Radiobutton(self.root, text="Every N keystrokes", variable=self.option_var, value="keystrokes").grid(row=0, column=2, padx=5, pady=5)

        self.n_var = tk.IntVar(value=5)
        ttk.Label(self.root, text="N keystrokes:").grid(row=1, column=0, padx=5, pady=5)
        ttk.Entry(self.root, textvariable=self.n_var, width=5).grid(row=1, column=1, padx=5, pady=5)

        ttk.Button(self.root, text="Clear Buffer", command=self.clear_buffer).grid(row=2, column=0, columnspan=3, pady=10)

        self.status_var = tk.StringVar()
        self.update_status()
        ttk.Label(self.root, textvariable=self.status_var).grid(row=3, column=0, columnspan=3, pady=5)

    def on_press(self, key):
        """키 입력 처리 핸들러"""
        if self.handle_special_keys(key):
            return

        if self.paused:
            return

        self.handle_key_input(key)
        self.update_layout_status()
        self.auto_predict()

    def handle_special_keys(self, key):
        """특수 키 처리(F1, F4)"""
        if key == keyboard.Key.f1:
            self.paused = not self.paused
            self.root.after(0, self.update_status)
            return True

        if key == keyboard.Key.f4:
            self.listener.stop()
            self.root.after(0, self.root.destroy)
            return True

        if key == keyboard.Key.space:
            self.handle_space()
            return True

        if key == keyboard.Key.backspace:
            self.handle_backspace()
            return True
        return False

    def handle_key_input(self, key):
        """일반 키 입력 처리"""
        try:
            if hasattr(key, 'char') and key.char:
                self.process_character(key.char)
        except AttributeError:
            pass

    def process_character(self, char):
        """문자 처리 및 버퍼 관리"""
        max_length = self.n_var.get()
        self.buffered_word += char
        
        # 버퍼 길이 제한
        if len(self.buffered_word) > max_length:
            self.buffered_word = self.buffered_word[-max_length:]
        
        self.keystroke_count += 1
        self.log_message(f"Key pressed: '{char}', Buffer: '{self.buffered_word}'")

    def auto_predict(self):
        """자동 예측 수행"""
        if self.option_var.get() == "keystrokes" and self.keystroke_count >= self.n_var.get():
            self.predict_and_switch()
            self.keystroke_count = 0

    def handle_space(self):
        """스페이스바 처리"""
        if self.option_var.get() == "space":
            self.predict_and_switch()
        else:
            self.buffered_word += " "
            self.keystroke_count += 1

    def handle_backspace(self):
        """백스페이스 처리"""
        if self.buffered_word:
            self.buffered_word = self.buffered_word[:-1]
            self.log_message(f"Backspace pressed. Buffer: '{self.buffered_word}'")

    def predict_and_switch(self):
        """언어 예측 및 레이아웃 전환"""
        prob = predict_language(self.buffered_word)
        

        if self.current_layout == "en" and prob <= THRESHOLD_LOW:
            print(f"Swich to Ko: {prob:.5f}")
            self.switch_layout("ko")
        elif self.current_layout == "ko" and prob >= THRESHOLD_HIGH:
            print(f"Swich to En: {prob:.5f}")
            self.switch_layout("en")

    def switch_layout(self, new_layout):
        """키보드 레이아웃 전환"""
        if new_layout == self.current_layout:
            return

        converter = conv_en2ko if new_layout == "ko" else conv_ko2en
        converted_text = converter(self.buffered_word)
        print(f"Buffer conversion: '{self.buffered_word}' -> '{converted_text}'")
        switch_language()
        self.buffered_word = converted_text
        self.current_layout = new_layout
        self.update_status()

    def update_layout_status(self):
        """키보드 레이아웃 상태 업데이트"""
        new_layout = check_language_mode()
        if new_layout != self.current_layout:
            self.current_layout = new_layout
            self.update_status()

    def update_status(self):
        """상태 표시 업데이트"""
        status = "English" if self.current_layout == "en" else "Korean"
        status += " (Paused)" if self.paused else ""
        self.status_var.set(f"Current Layout: {status}")

    def clear_buffer(self):
        """버퍼 초기화"""
        self.buffered_word = ""
        self.keystroke_count = 0
        self.log_message("Buffer cleared via button.")

    def log_message(self, message):
        """로그 메시지 기록"""
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        with open(log_file, 'a', encoding='utf-8') as f:
            f.write(f"{timestamp} - {message}\n")

if __name__ == "__main__":
    root = tk.Tk()
    app = KeyboardPredictorApp(root)
    root.mainloop()

Swich to Ko: 0.05818
Buffer conversion: 'nglish rmf' -> 'ㅜ히ㅑ노 글'
Swich to En: 0.95645
Buffer conversion: 'may I talk' -> 'may I talk'
